In [746]:
import os
import pandas as pd
import numpy as np

# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF

(Kolumny: OD M DO AB. Wiersze: Od 3 do 18 włącznie)
(wiersze 30 do 93 włącznie)
Wiersze 255 – 275 włącznie
Wiersze 279 – 287 włącznie (dane w %)
Wiersze 290 – 293 włącznie

In [747]:
# PEŁNY DF
full_row_ranges = [
    range(290, 293+1),  # Current financial liabilities, Non-current financial liabilities, Financial liabilities (total)
    range(278, 283+1),  # Return on equity (ROE), Return on assets (ROA), Leverage (EM), Asset utilization (AU)
    range(260, 275+1),  # Working capital ratio, Current ratio, Quick ratio, Cash ratio, Receivables turnover, Inventory turnover, The operating cycle, Rotation commitments, Cash conversion cycle, Rotation assets, Rotation of assets, Assets ratio, Debt ratio, Debt service ratio, Rate debt security
    range(29, 59+1),    # ASSETS, Non-current assets, Property, plant and equipment, Exploration for and evaluation of mineral resources, Intangible assets, Goodwill, Investment property, Right-of-use assets, Investment in affiliates, Non-current financial assets, Non-current loans and receivables, Deferred income tax, Non-current deferred charges and accruals, Non-current derivative instruments, Other non-current assets, Current assets, Inventories, Current intangible assets, Biological assets, Trade receivables, Loans and other receivables, Financial assets, Cash and cash equivalents, Accruals, Assets from current tax, Derivative instruments, Other assets, Assets held for sale and discontinuing operations, Called up capital, Own shares
    range(60, 93+1),    # EQUITY & LIABILITIES, Equity shareholders of the parent, Share capital, Called up share capital, Treasury shares, Supplementary capital, Valuation and exchange differences, Other capitals, Retained earnings / accumulated losses, Non-controlling interests, Non-current liabilities, Non-current liabilities from derivatives, Non-current loans and borrowings, Non-current liabilities from bonds, Non-current liabilities from finance leases, Non-current trade payables, Long-term provision for employee benefits, Deferred tax liabilities, Non-current provision, Other non-current liabilities, Non-current accruals (liability), Current liabilities, Liabilities from derivatives, Financial liabilities (loans and borrowings), Bond liabilities, Liabilities from finance leases, Trade payables, Employee benefits, Current tax liabilities, Provisions, Other liabilities, Accruals (liability), Liabilities related to assets held for sale and discontinued operations
    range(2, 22+1),     # End of period, Revenues from sales, Operating profit/loss, Net profit/loss attributable to equity holders of the parent, Total comprehensive income attributable to equity holders, Depreciation, Cash flow from operating activities, Cash flow from investing activities, Cash flow from financing activities, Non-current assets, Current assets, Equity shareholders of the parent, Non-current liabilities, Current liabilities, Number of shares (‘000), Earnings per share, Total comprehensive income per share, Book value per share
]

**OGRANICZONY DF** (testować, który sobie lepiej radzi)
#TODO, weryfikacja wszystkich wskaźników i wybór tych najbardziej odpowiadających
Informacje dodatkowe:
    Current financial liabilities
    Non-current financial liabilities
    Financial liabilities (total)

Wskaźniki DuPonta: (do przejrzenia)
    Return on equity (ROE)
    Return on assets (ROA)
    Leverage (EM)
    Net profit margin
    Asset utilization (AU)

Wskaźniki: (do przejrzenia)
    Current ratio
    Quick ratio
    Cash ratio
    Receivables turnover
    Inventory turnover
    The operating cycle
    Rotation commitments
    Cash conversion cycle
    Rotation assets
    Rotation of assets
    Assets ratio
    Debt ratio
    Debt service ratio
    Rate debt security

Podstawowe informacje:
Data bilansu

**Aktywa**
Non-current assets
Current assets

**Pasywa** (Suma Equity shareholders of the parent, Non-current liabilities i Current liabilities)
Equity shareholders of the parent
Non-current liabilities
Current liabilities

+ feature engineering wcześniejszego kwartału, półrocza, roku, dwóch lat na każdy z ważnych elementów

## Dodawanie wybranych kolumn do pustego DataFrame
df_empty = df_source[kolumny_do_skopiowania]

Suma wszystkich kolumn to:
3 + 4 + 15 + 30 + 33 + 21 = 106

Dodając jedną dodatkową kolumnę na nazwę pliku, otrzymujemy:
106 + 1 = 107 kolumn

# PREPARATION

In [748]:
FOLDER_PATH = 'dataa/'

In [749]:
if os.path.exists(FOLDER_PATH):
    print("Folder exists. Proceeding with file processing.")
else:
    print("Folder does not exist. Check the path.")

Folder exists. Proceeding with file processing.


In [750]:
def count_files_in_folder(folder_path):
    file_count = 0

    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            file_count += 1
    
    return file_count

print(f"Files in folder: {count_files_in_folder(FOLDER_PATH)}.")

Files in folder: 1.


# 'INFO' DATA

In [751]:
def process_company_sector(folder_path):
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                with pd.ExcelFile(file_path) as xls:
                    company_name = pd.read_excel(xls, 'Info', usecols="B", skiprows=1, nrows=1).values[0][0]
                    sector = pd.read_excel(xls, 'Info', usecols="E", skiprows=19, nrows=1).values[0][0]
                
                result = {
                    'filename': filename,
                    'Company Name': company_name,
                    'Sector': sector
                }
                results.append(result)
                
                print(f"Dane z {filename} zostały przetworzone.")
            except Exception as e:
                print(f"Błąd przy przetwarzaniu pliku {filename}: {e}")

    results_df = pd.DataFrame(results)
    return results_df

In [752]:
process_company_sector(FOLDER_PATH)

Dane z ZYWIEC.xlsx zostały przetworzone.


,filename,Company Name,Sector
0,ZYWIEC.xlsx,Grupa Żywiec SA,napoje


# QS DATA

In [753]:
def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    row_ranges = [
        range(2, 19),  # Rows 3 to 18 inclusive, zero-indexed
        range(30, 94),  # Rows 30 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 288),  # Rows 279 to 287 inclusive
        range(289, 294)  # Rows 290 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(file_path, sheet_name='QS', usecols=[2], header=None)

                extracted_data = []
                for row_range in row_ranges:
                    range_data = data.iloc[row_range].squeeze().tolist()
                    concatenated_data = ', '.join(map(str, range_data))
                    extracted_data.append(concatenated_data)

                new_row = {}
                for item in extracted_data:
                    for entry in item.split(', '):
                        new_row[entry.strip()] = None  # dummy value

                new_df = pd.DataFrame([new_row])
                new_df['filename'] = filename

                all_data.append(new_df)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.concat(all_data, ignore_index=True)
    return results_df

df_results = process_excel_data(FOLDER_PATH)
df_results

,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,Cash flow from financing activities,...,Leverage (EM),Asset utilization (AU),Load gross profit,Load operating profit,EBITDA margin,EBITDA,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total),filename
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,ZYWIEC.xlsx


In [754]:
def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    row_ranges = [
        range(2, 23),   # Rows 3 to 22 inclusive, zero-indexed
        range(30, 60),  # Rows 31 to 59 inclusive
        range(60, 94),  # Rows 61 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 284),  # Rows 279 to 283 inclusive
        range(290, 294)   # Rows 291 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(file_path, sheet_name='QS', usecols=[2], header=None)

                extracted_data = []
                for row_range in row_ranges:
                    range_data = data.iloc[row_range].squeeze().tolist()
                    concatenated_data = ', '.join(map(str, range_data))
                    extracted_data.append(concatenated_data)

                new_row = {}
                for item in extracted_data:
                    for entry in item.split(', '):
                        new_row[entry.strip()] = None  # dummy value

                new_df = pd.DataFrame([new_row])
                new_df['filename'] = filename

                all_data.append(new_df)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.concat(all_data, ignore_index=True)
    return results_df

# Podaj swoją ścieżkę folderu
df_results = process_excel_data(FOLDER_PATH)
df_results

,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,Cash flow from financing activities,...,Debt ratio,Debt service ratio,Rate debt security,Leverage (EM),Asset utilization (AU),Load gross profit,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total),filename
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,ZYWIEC.xlsx


In [755]:
# DZIAŁA PRAWIE DOBRZE
def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    # Zakresy wierszy, które chcemy wyciągnąć (przesunięte o jeden w dół, ponieważ indeksowanie jest od zera)
    row_ranges = [
        range(2, 23),   # Rows 3 to 22 inclusive, zero-indexed
        range(30, 60),  # Rows 31 to 59 inclusive
        range(60, 94),  # Rows 61 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 284),  # Rows 279 to 283 inclusive
        range(290, 294)   # Rows 291 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {filename}")  # Debugging line
            try:
                # Wczytujemy cały arkusz, aby mieć dostęp do kolumn C do CX
                data = pd.read_excel(file_path, sheet_name='QS', header=None)

                # Przechodzimy przez wszystkie zakresy wierszy i wyciągamy dane
                for row_range in row_ranges:
                    # Pobieramy wartości z kolumny C jako nagłówki
                    headers = data.iloc[list(row_range), 2].tolist()

                    # Pobieramy dane z kolumn D do CX dla bieżącego zakresu wierszy (indeksowanie od 3 do 102)
                    range_data = data.iloc[list(row_range), 3:102]

                    # Zamiana zer na NaN
                    range_data = range_data.replace(0, np.nan)

                    # Transponowanie danych, aby daty były w pierwszej kolumnie
                    transposed_data = range_data.transpose()
                    transposed_data.columns = headers
                    transposed_data['filename'] = filename


                    all_data.append(transposed_data)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    if all_data:
        results_df = pd.concat(all_data, ignore_index=True)

        # Sprawdzenie i usunięcie wierszy, które są w całości NaN (poza kolumną filename)
        results_df = results_df.dropna(how='all', subset=[col for col in results_df.columns if col != 'filename'])

        # Sprawdzenie i usunięcie kolumny z nazwą NaN
        results_df = results_df.loc[:, ~results_df.columns.isna()]

        return results_df
    else:
        print("No data to concatenate")
        return pd.DataFrame()

# Podaj swoją ścieżkę folderu
df_results = process_excel_data(FOLDER_PATH)
df_results

Processing file: ZYWIEC.xlsx


C:\Users\ksobc\AppData\Local\Temp\ipykernel_22960\1254272257.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  range_data = range_data.replace(0, np.nan)
C:\Users\ksobc\AppData\Local\Temp\ipykernel_22960\1254272257.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  range_data = range_data.replace(0, np.nan)


,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,Cash flow from financing activities,...,Assets ratio,Debt ratio,Debt service ratio,Rate debt security,Leverage (EM),Asset utilization (AU),Load gross profit,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total)
0,1998-03-31,1998-01-01,126003,11171,5812,NaN,11442,34508,-17981,-17358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-06-30,1998-04-01,142773,50830,33203,NaN,12009,33829,-12482,-21016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998-09-30,1998-07-01,222098,61141,41145,NaN,13995,52800,-12265,-17596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998-12-31,1998-10-01,154110,27972,18590,NaN,12549,45540,-1632209,1578310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1999-03-31,1999-01-01,168924,31190,18760,NaN,12387,95255,-138856,28489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,863122.0,NaN,863122.0
590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1094585.0,NaN,1094585.0
591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,613619.0,540000.0,1153619.0
592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96674.0,975000.0,1071674.0


In [756]:
df_results.columns.tolist()

['End of period',
 'Start of period',
 'Revenues from sales',
 'Operating profit/loss',
 'Net profit/loss attributable to equity holders of the parent',
 'Total comprehensive income attributable to equity holders',
 'Depreciation',
 'Cash flow from operating activities',
 'Cash flow from investing activities',
 'Cash flow from financing activities',
 'Aktywa',
 'Non-current assets',
 'Current assets',
 'Equity shareholders of the parent',
 'Non-current liabilities',
 'Current liabilities',
 "Number of shares ('000)",
 'Earnings per share',
 'Total comprehensive income per share',
 'Book value per share',
 'Accounting standard',
 'filename',
 'Property, plant and equipment',
 'Exploration for and evaluation of mineral resources',
 'Intangible assets',
 'Goodwill',
 'Investment property',
 'Right-of-use assets',
 'Investment in affiliates',
 'Non-current financial assets',
 'Non-current loans and receivables',
 'Deferred income tax',
 'Non-current deferred charges and accruals',
 'Non-cu

In [757]:
df_cleaned_columns = df_results.dropna(how='all', axis=1)

In [758]:
df_cleaned_columns = df_results.drop(columns='Start of period')

In [759]:
df_cleaned_columns

,End of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,Cash flow from financing activities,Aktywa,...,Assets ratio,Debt ratio,Debt service ratio,Rate debt security,Leverage (EM),Asset utilization (AU),Load gross profit,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total)
0,1998-03-31,126003,11171,5812,NaN,11442,34508,-17981,-17358,383455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-06-30,142773,50830,33203,NaN,12009,33829,-12482,-21016,456063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998-09-30,222098,61141,41145,NaN,13995,52800,-12265,-17596,449272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998-12-31,154110,27972,18590,NaN,12549,45540,-1632209,1578310,2051117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1999-03-31,168924,31190,18760,NaN,12387,95255,-138856,28489,2183358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,863122.0,NaN,863122.0
590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1094585.0,NaN,1094585.0
591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,613619.0,540000.0,1153619.0
592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96674.0,975000.0,1071674.0


In [760]:
df_cleaned_columns.columns.tolist()

['End of period',
 'Revenues from sales',
 'Operating profit/loss',
 'Net profit/loss attributable to equity holders of the parent',
 'Total comprehensive income attributable to equity holders',
 'Depreciation',
 'Cash flow from operating activities',
 'Cash flow from investing activities',
 'Cash flow from financing activities',
 'Aktywa',
 'Non-current assets',
 'Current assets',
 'Equity shareholders of the parent',
 'Non-current liabilities',
 'Current liabilities',
 "Number of shares ('000)",
 'Earnings per share',
 'Total comprehensive income per share',
 'Book value per share',
 'Accounting standard',
 'filename',
 'Property, plant and equipment',
 'Exploration for and evaluation of mineral resources',
 'Intangible assets',
 'Goodwill',
 'Investment property',
 'Right-of-use assets',
 'Investment in affiliates',
 'Non-current financial assets',
 'Non-current loans and receivables',
 'Deferred income tax',
 'Non-current deferred charges and accruals',
 'Non-current derivative ins